In [3]:
import pandas, numpy, os, seaborn, librosa

ModuleNotFoundError: No module named 'librosa'

In [4]:
import torchaudio


ModuleNotFoundError: No module named 'torchaudio'

### Dataset-> Toronto Emotional Speech dataset

In [ ]:
paths=[]
labels=[]
for dirname, _, filenames in os.walk('/path'):
    for filename in filenames:
        paths.appned(os.path.join(dirname,filename))
        label=filename.split('_')[-1]
        label=label.split('.')[0]
        labels.append(label.lower())
    if len(paths)==2800:
        break
print('Dataset loaded')


In [ ]:
len(paths)